## DCGAN implementation in Keras

Inspired by jacobgil's implementation: [https://github.com/jacobgil/keras-dcgan/](https://github.com/jacobgil/keras-dcgan) and [http://bamos.github.io/2016/08/09/deep-completion/](http://bamos.github.io/2016/08/09/deep-completion/)

In [1]:
from __future__ import print_function,division
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation,Flatten
# from keras.layers import Deconvolution2D as Deconv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.datasets import mnist
from tqdm import tqdm_notebook 
import numpy as np
from PIL import Image
import tensorflow as tf
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


## Models

In [2]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url = "http://bamos.github.io/data/2016-08-09/gen-architecture.png")

In [3]:
Image(url = "http://bamos.github.io/data/2016-08-09/discrim-architecture.png")

In [21]:
# TODO(jkg): add better descriptions of models

def keras_generator_model():
    '''generate images
    '''
    model = Sequential()
    ##########
    model.add(Dense(input_dim = 100, output_dim = 1024*4*4))
    model.add(Activation('tanh'))
    print(model.output_shape)
    model.add(Reshape((1024,4,4),input_shape=(1024*4*4,)))
    print(model.output_shape)
    model.add(UpSampling2D(size=(4,4)))
    model.add(Convolution2D(512,5,5,subsample=(2,2),border_mode = 'same'))
    model.add(BatchNormalization(mode=2,axis=3,epsilon=1e-5))
    model.add(Activation('tanh'))
    print(model.output_shape)
    ##########
    model.add(UpSampling2D(size=(4,4)))
    model.add(Convolution2D(256,5,5,subsample=(2,2),border_mode = 'same'))
    model.add(BatchNormalization(mode=2,axis=3,epsilon=1e-5))
    model.add(Activation('tanh'))
    print(model.output_shape)
    ##########
    model.add(UpSampling2D(size=(4,4)))
    model.add(Convolution2D(128,5,5,subsample=(2,2),border_mode = 'same'))
    model.add(BatchNormalization(mode=2,axis=3,epsilon=1e-5))
    model.add(Activation('tanh'))
    print(model.output_shape)
    ##########
    model.add(UpSampling2D(size=(4,4)))
    model.add(Convolution2D(3,5,5,subsample=(2,2),border_mode = 'same'))
    model.add(BatchNormalization(mode=2,axis=3,epsilon=1e-5))
    model.add(Activation('tanh'))
    print(model.output_shape)
    return model

def keras_discriminator_model():
    '''discriminate images
    '''
    model = Sequential()
    model.add(Convolution2D(64, 5, 5,border_mode = 'same',input_shape = (3,64,64)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    print(model.output_shape)
    ##########
    model.add(Convolution2D(128, 5, 5,border_mode = 'same'))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    print(model.output_shape)
    ###########
    model.add(Convolution2D(256, 5, 5,border_mode = 'same'))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    print(model.output_shape)
    ###########
    model.add(Convolution2D(512, 5, 5,border_mode = 'same'))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    print(model.output_shape)
    model.add(Flatten())
    print(model.output_shape)
    model.add(Dense(1024))
    print(model.output_shape)
    model.add(Activation('tanh'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    print(model.output_shape)
    return model

def generator_and_discriminator(gen_model,dis_model):
    '''stack the generator model on top of the discriminator model
    only allow the generator model to be trainable
    '''
    model = Sequential()
    model.add(gen_model)
    dis_model.trainable = False
    model.add(dis_model)
    return model

In [22]:
gen = keras_generator_model()
gen.output_shape
print('----------')
desc = keras_discriminator_model()

(None, 16384)
(None, 1024, 4, 4)
(None, 512, 8, 8)
(None, 256, 16, 16)
(None, 128, 32, 32)
(None, 3, 64, 64)
----------
(None, 64, 32, 32)
(None, 128, 16, 16)
(None, 256, 8, 8)
(None, 512, 4, 4)
(None, 8192)
(None, 1024)
(None, 1)


In [ ]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[2:]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[0, :, :]
    return image

## print nice images
def diagnostic_image(gen,epoch):
    batch_size = 25
    noise = np.zeros((batch_size*20,100))
    for i in range(noise.shape[0]):
        noise[i,:] = np.random.uniform(-1,1,100)
    generated_images = gen.predict(noise,verbose=0)
    image = combine_images(generated_images)
    image = image * 127.5 + 127.5
    Image.fromarray(image.astype(np.uint8)).save("epoch_{}.png".format(epoch))

## Testing model w/ MNIST

In [ ]:
# # Normalize and Reshape data
# (X_train,y_train),(X_test,y_test) = mnist.load_data()

# # why do this instead of convert to [0,1] - center and scale not just scale
# X_train_n = (X_train.astype(np.float32) - 127.5)/127.5
# # X_train = X_train / 255 <-

# X_train_n = X_train_n.reshape((X_train.shape[0],1,X_train.shape[1],X_train.shape[2]))
# # new shape should be (60000,1,28,28) 1 channel image 28x28 px
# X_train_n.shape

## Testing model w/ actual art

In [ ]:
import cPickle as pickle

with open('shrunk_baroque.pkl','rb') as f:
    imgs = pickle.load(f)
imgs.shape

In [ ]:
# want(samples,3,100,100)
imgs = imgs * 255
imgs = imgs.astype(np.uint8)

In [ ]:
img = Image.fromarray(imgs[4])
img.save('bartest.png')

In [ ]:
imgs_n = (imgs.astype(np.float32)-127.5)/127.5
imgs_n = imgs_n.reshape((imgs.shape[0],3,imgs.shape[1],imgs.shape[2]))

In [ ]:
imgs_n.shape
X_train_n = imgs_n

In [ ]:
checkpoint_folder = 'chk/'
batch_size = 32
epochs = 100
batchs_per_epoch = int(X_train_n.shape[0]/batch_size)

In [23]:
# Instantiate Models
desc = keras_discriminator_model()
gen = keras_generator_model()
d_on_g = generator_and_discriminator(gen,desc)

d_op = SGD(lr = 5e-4,momentum = 0.9,nesterov=True)
g_op = SGD(lr = 5e-4,momentum = 0.9,nesterov=True)

gen.compile(loss='binary_crossentropy',optimizer = "SGD")
print(gen.output_shape)
d_on_g.compile(loss='binary_crossentropy',optimizer = g_op)

desc.trainable = True # necessary??
desc.compile(loss='binary_crossentropy',optimizer= d_op)

(None, 64, 32, 32)
(None, 128, 16, 16)
(None, 256, 8, 8)
(None, 512, 4, 4)
(None, 8192)
(None, 1024)
(None, 1)
(None, 16384)
(None, 1024, 4, 4)
(None, 512, 8, 8)
(None, 256, 16, 16)
(None, 128, 32, 32)
(None, 3, 64, 64)
(None, 3, 64, 64)


In [ ]:
noise = np.zeros((batch_size,100))

In [ ]:
epoch_loss = []
for epoch in tqdm_notebook(range(epochs),desc = 'epoch'):
    batch_loss = []
    for batch_index in tqdm_notebook(range(batchs_per_epoch),desc = 'batch',leave=False):
        #build noise up
        for sample in range(batch_size):
            noise[sample,:] = np.random.uniform(-1,1,100)
        
        image_set = X_train_n[batch_index*batch_size:(batch_index+1)*batch_size]
        # print(image_set.shape)
        # feed noise into generator network
        gen_images = gen.predict(noise,verbose=0)
        # print(gen_images.shape)
        # now we have data for our desc network
        X_desc = np.concatenate([image_set,gen_images],axis=0)
        y = [1] * batch_size + [0] * batch_size
        
        # train desc
        d_loss = desc.train_on_batch(X_desc,y)
        
        #build noise up
        for sample in range(batch_size):
            noise[sample,:] = np.random.uniform(-1,1,100)
            
        # train generator on improved desc
        desc.trainable = False
        g_loss = d_on_g.train_on_batch(noise,[1] * batch_size)
        desc.trainable = True
        
        if(batch_index+1 == batchs_per_epoch):
            gen.save_weights(checkpoint_folder+'g_weights.h5',overwrite=True)
            desc.save_weights(checkpoint_folder+'d_weights.h5',overwrite=True)
        batch_loss.append([d_loss,g_loss])
    #if epoch % 5 == 0:
    diagnostic_image(gen,epoch)
    avg_d = np.mean([x[0] for x in batch_loss])
    avg_g = np.mean([x[1] for x in batch_loss])
    print('avg desc loss {} || avg gen loss {}'.format(avg_d,avg_g))
    epoch_loss.append([avg_d,avg_g])

![all_img.png](all_img.png)